# Simplified Multi-Domain Assessment

Intention of this section is to assess the usefulness of *non prioritised* weighting in multi domain assessment. Effectively applying MTMF with individual / subsets of metrics across domains. 

## 1-Metric Matrix

$$
H = [H_0, H_1, \cdots, H_M]^T = 
\left[ 
\begin{array}{cccc}
1 & 0 & \cdots  & 0 \\
0 & 1 & \cdots  & 0 \\
0 & 0 & \cdots  & 1
\end{array}
\right]$$

$$ \begin{align}
  \Delta T_{ix} &= \frac{\sum_{j\neq x}\left( \overline{T_{i,j}}^{\forall t}\right)}{N-1} \\
  \Delta T_{ix}^- &= \frac{\sum_{j\neq x} \Delta T_{ij}}{N-1}
\end{align}
$$

The Performance ($P$) and False Positive Performance ($P^-$) of the system can then be described as

$$ H . 
\left[
\begin{array}{c}
\Delta T_{ix}^0 \\
\Delta T_{ix}^1 \\
\cdots \\
\Delta T_{ix}^M
\end{array}
\right] = P$$

In [42]:
import itertools
min_emphasis = 0
min_sum=1
max_emphasis = 2
max_sum=1
M=7
if max_sum is None:
    max_sum = np.inf
combinations = itertools.ifilter(np.any,
                      itertools.product(
                          xrange(
                              min_emphasis, max_emphasis
                          ), repeat=M
                      )
                      )
combinations = itertools.ifilter(lambda v: min_sum <= np.sum(v) <= max_sum,
                                combinations)
combinations = sorted(combinations, key=lambda l: sum(map(abs, l)))
combinations

[(0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 1, 0, 0),
 (0, 0, 0, 1, 0, 0, 0),
 (0, 0, 1, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0, 0)]

In [34]:
combinations

[]

In [47]:
s = "ANYTHING"
s.startswith('ANY')

True

In [22]:
list(xrange(0,2))

[0, 1]